In [2]:
import numpy as np

In [3]:
import sys,os
import numpy as np

from quspin.basis import spin_basis_1d, spin_basis_general
from quspin.operators import hamiltonian,quantum_operator

os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='1' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='12' # set number of MKL threads to run in parallel

In [4]:
Lx,Ly = 3, 4
N_2d = Lx*Ly
Jz, Jx, Jy, h = [1, -0.3, -0.3, 1]
s = np.arange(N_2d)
x = s%Lx 
y = s//Lx 
T_x = (x+1)%Lx + Lx*y 
T_y = x +Lx*((y+1)%Ly) 

In [5]:
J_zz = [[1,i,T_x[i]] for i in range(N_2d)]+[[1,i,T_y[i]] for i in range(N_2d)] 
h_list = [[-1.0,i] for i in range(N_2d)]
ops_dict = dict(Jz=[["zz", J_zz]], # J for z
                Jx=[["xx",J_zz]],
                Jy=[["yy",J_zz]], # J for x and y
                h=[["z",h_list]])
M_list = [[1.0/N_2d,i] for i in range(N_2d)]

basis_2d = spin_basis_general(N_2d ,pauli=False)
_H = quantum_operator(ops_dict,basis=basis_2d, dtype=np.float64, check_symm=False)
M = hamiltonian([["z",M_list]],[],basis=basis_2d, dtype=np.float64)
params_dict=dict(Jz=Jz, Jx=Jx,Jy=Jy,h=h)

Hermiticity check passed!
Hermiticity check passed!
Symmetry checks passed!


In [6]:
Hs = _H.tohamiltonian(params_dict)
H = Hs.toarray()

In [7]:
E, V = np.linalg.eigh(H)

In [ ]:
T = 1
M_np = M.toarray()
beta = 1/T
Z = np.exp(-beta*E).sum()
M_mean = np.einsum("n,in,jn,ij->", np.exp(-beta*E), V, V, M_np) / Z
M2_mean = np.einsum("n,in,jn,ij->", np.exp(-beta*E), V, V, M_np@M_np) / Z
E_mean = (E*np.exp(-beta*E)).sum() / Z
E2_mean = ((E*E)*np.exp(-beta*E)).sum() / Z

In [214]:
print(params_dict)
print(f"T               = {T}")
print(f"E               = {E_mean / L}")
print(f"C               = {(E2_mean - E_mean**2)*(beta**2)/L}")
print(f"M               = {M_mean}")
print(f"M^2             = {M2_mean}")

{'Jz': 1, 'Jx': -0.3, 'Jy': -0.3, 'h': 1}
T               = 0.3
E               = -0.2730339836265664
C               = 0.13699753184107943
M               = 0.20530285681405502
M^2             = 0.05188517444027588


In [215]:
print(f"Chi             = {(0.20530285681405502 - 0.20239124658421112)/0.01}")

Chi             = 0.2911610229843903
